## 1 - Summary Statistics

In [77]:
import pandas as pd 
import numpy as np

# columns: date, SPY ret, TB1M ret
data = pd.read_excel('barnstable_analysis_data.xlsx',sheet_name='data')
data['date'] = pd.to_datetime(data['date'])
data_65_99 = data[(data['date'] >= '1965-01-01') & (data['date'] <= '1999-12-31')]
data_00 = data[(data['date'] >= '2000-01-01') & (data['date'] <= '2024-12-31')]
data_26 = data[(data['date'] >= '1926-01-01') & (data['date'] <= '2024-12-31')]

In [ ]:
## Summary Statistics 
# r^m => market return, r^f => risk free rate, r~ => excess returns

def ret_calc(df): 
    out = df.copy()

    # df creation
    out['r_m'] = out['SPX']
    mm,vm = 12 * out['r_m'].mean(), (12)**.5 * out['r_m'].std()
    out['r_f'] = out['TB1M']
    mf,vf = 12 * out['r_f'].mean(), (12)**.5 * out['r_f'].std()
    out['xs'] = out['r_m'] - out['r_f']
    mxs,vxs = 12 *  out['xs'].mean(), (12)**.5 * out['xs'].std()
    out['lr_m'] = np.log(1 + out['SPX'])
    lmm,lvm = 12 * out['lr_m'].mean(), (12)**.5 *  out['lr_m'].std()
    out['lr_f'] = np.log(1 + out['TB1M'])
    lmf,lvf = 12 * out['lr_f'].mean(), (12)**.5 * out['lr_f'].std()   
    out['lxs'] = out['lr_m'] - out['lr_f']
    lmxs,lvxs = 12 * out['lxs'].mean(), (12)**.5 * out['lxs'].std()

    stats = [mm,mf,mxs,lmm,lmf,lmxs,vm,vf,vxs,lvm,lvf,lvxs]
    return stats,out

# 1926-2024
stats_26,frame_26 = ret_calc(data_26)

# 2000-2024
stats_00,frame_00 = ret_calc(data_00)

# 1965-1999
stats_65_99,frame_65_99 = ret_calc(data_65_99)

# df creation
results = pd.DataFrame()
results['1965-1999 Mean'] = stats_65_99[0:6]
results['1965-1999 Vol'] = stats_65_99[6:]
results['2000-2024 Mean'] = stats_65_99[0:6]
results['2000-2024 Vol'] = stats_65_99[6:]
results['1926-2024 Mean'] = stats_65_99[0:6]
results['1926-2024 Vol'] = stats_65_99[6:12]
results.index = ['Real','Risk Free','Excess','Log Real','Log Risk Free', 'Log Excess']
print(results.round(4))

               1965-1999 Mean  1965-1999 Vol  2000-2024 Mean  2000-2024 Vol  \
Real                   0.1294         0.1494          0.1294         0.1494   
Risk Free              0.0615         0.0072          0.0615         0.0072   
Excess                 0.0687         0.1502          0.0687         0.1502   
Log Real               0.1176         0.1496          0.1176         0.1496   
Log Risk Free          0.0613         0.0071          0.0613         0.0071   
Log Excess             0.0571         0.1504          0.0571         0.1504   

               1926-2024 Mean  1926-2024 Vol  
Real                   0.1294         0.1494  
Risk Free              0.0615         0.0072  
Excess                 0.0687         0.1502  
Log Real               0.1176         0.1496  
Log Risk Free          0.0613         0.0071  
Log Excess             0.0571         0.1504  


## Probability of Underperformance